## Chunked queries: Efficient Data Retrieval

When dealing with large result sets, fetching the entire data at once can lead to excessive memory usage and slower performance. 
Fortunately, there is a solution called "chunked queries" that allows us to retrieve data in smaller, manageable chunks. 
By employing this technique, we can optimize memory usage and significantly improve query performance.

Chunked queries are particularly useful when working with datasets that contain millions of data points. 
Rather than requesting the entire result set in one go, we can specify a maximum chunk size to split the data into smaller portions. 

It's important to note that the optimal chunk size may vary depending on the specific query.
While it may seem intuitive that a smaller chunk size would result in faster query execution, that's not always the case. In fact, setting the chunk size too small can introduce overhead by generating a large number of requests to the database. 


In [ ]:
import pandas as pd
from lsst_efd_client import EfdClient
client = EfdClient('usdf_efd')

To enable chunked responses in your query, you can simply set the chunked parameter to `True`. 
Additionally, you can specify the `chunk_size` parameter, which determines the maximum number of data points in each chunk. 
By default, the chunk size is set to 1000, but you can adjust it based on your specific requirements.

Let's consider an example where we need to query the `lsst.sal.MTM1M3.forceActuatorData` topic. 

In [ ]:
await client.get_schema("lsst.sal.MTM1M3.forceActuatorData")


This particular topic contains a massive payload, making it inefficient to retrieve all the data using a `SELECT * FROM ...` statement. 
Instead, it is recommended to select only the fields of interest. 
This significantly improves the query speed.

The topic schema reveals the presence of array fields. 
In this topic, each element within the array corresponds to a distinct force actuator. 
Specifically, there are 156 actuators responsible for exerting forces in the `x` direction.

In [ ]:
fields =  ", ".join([f"xForce{i}" for i in range(156)])

In [ ]:
query = f'''SELECT {fields} FROM "lsst.sal.MTM1M3.forceActuatorData" WHERE time > now()-1h'''
query

By implementing chunked queries with the appropriate configuration, we can retrieve a dataframe with approximately 30 million data points in less than a minute.

This approach leverages the power of chunked responses and targeted data selection to achieve efficient and timely data retrieval.

In [ ]:
chunks = await client.influx_client.query(query, chunked=True, chunk_size=1000)
df = pd.concat([chunk async for chunk in chunks])

In [ ]:
df.head()

In [ ]:
df.size